# Open Banking Directory explorer
Use this notebook to query the Open Banking directory for sandboxes

NB: You need provide your own copy of the latest ASPSPs.json

In [1]:
from pathlib import Path
import json

aspsps_file = Path("./local/ASPSPs.json")
assert aspsps_file.is_file(), "You need to download the data (* Fintech > OBIE > Open Banking Get ASPSPs > Results)"

aspsps = None
with aspsps_file.open('r') as handle:
    data = handle.read()
    aspsps = json.loads(data)

resources = [resource for resource in aspsps['Resources']]

In [2]:
print('List of ASPSPs that are active:')
count = 0
for resource in resources:
    name = resource['urn:openbanking:organisation:1.0']['OrganisationCommonName']
    ob_id = resource['urn:openbanking:organisation:1.0']['OBOrganisationId']
    has_auth = 'AuthorisationServers' in resource
    is_active = any(auth['Active'] for auth in resource["urn:openbanking:competentauthorityclaims:1.0"]['Authorisations'])
    
    if is_active:
        print(ob_id,'\t', name, '\t', has_auth)
        count += 1

print(count, ' banks')

List of ASPSPs that are active:
0015800001041RHAAY 	 Ozone Financial Technology Limited 	 True
0015800001041RbAAI 	 Open Banking Limited (A) 	 True
0015800000jf9VgAAI 	 AIB Group (UK) plc 	 True
00158000016i44JAAQ 	 HSBC UK Bank Plc 	 False
00158000016i44VAAQ 	 Starling Bank Limited 	 False
0015800000jfAW1AAM 	 Barclays Bank UK Plc 	 True
0015800000jfPKvAAM 	 Bank of Scotland PLC 	 True
0015800000jf8aKAAQ 	 Nationwide Building Society 	 True
0015800000jfFGuAAM 	 Santander UK PLC 	 True
0015800000ti1PbAAI 	 Coutts & Company 	 True
0015800000jf9GgAAI 	 Lloyds Bank PLC 	 True
0015800000jfQ9aAAE 	 Bank of Ireland (UK) Plc 	 True
0015800000jeox1AAA 	 HSBC Bank plc 	 False
0015800000jf7AeAAI 	 Northern Bank Limited t/a Danske Bank 	 True
0015800000jfxrpAAA 	 Ulster Bank Ltd 	 False
0015800000jfwB4AAI 	 The Royal Bank of Scotland Plc 	 True
0015800000jfwxXAAQ 	 National Westminster Bank Plc 	 True
0015800000jeukaAAA 	 Marks & Spencer Financial Services Plc 	 False
0015800001041QVAAY 	 Clydesd

In [3]:
print('List of ASPSPs that are active and have an Authorisation Server defined:')
count = 0
for resource in resources:
    name = resource['urn:openbanking:organisation:1.0']['OrganisationCommonName']
    ob_id = resource['urn:openbanking:organisation:1.0']['OBOrganisationId']
    has_auth = 'AuthorisationServers' in resource
    is_active = any(auth['Active'] for auth in resource["urn:openbanking:competentauthorityclaims:1.0"]['Authorisations'])
    
    if is_active and has_auth:
        for auth in resource['AuthorisationServers']:
            friendly_name = auth.get('CustomerFriendlyName')
            portal_url = auth.get('DeveloperPortalUri')
            config_url = auth.get('OpenIDConfigEndPointUri')
            
            print(f"{ob_id}\t{friendly_name}\t{portal_url}\t{config_url}\t({name})")
            count += 1

print(count, ' Auth servers')

List of ASPSPs that are active and have an Authorisation Server defined:
0015800001041RHAAY	EU Modelo Bank	https://o3bank.com	https://eumodelobankauth2018.o3bank.co.uk:4103/.well-known/openid-configuration	(Ozone Financial Technology Limited)
0015800001041RHAAY	EU Referenco Bank	https://o3bank.com	https://eureferencobankauth2018.o3bank.co.uk:4104/.well-known/openid-configuration	(Ozone Financial Technology Limited)
0015800001041RHAAY	Ozone Bank 2	https://o3bank.com	https://ob19-auth2-ui.o3bank.co.uk/.well-known/openid-configuration	(Ozone Financial Technology Limited)
0015800001041RHAAY	Ozone - Bank 1	https://o3bank.com/	https://ob19-auth1-ui.o3bank.co.uk/.well-known/openid-configuration	(Ozone Financial Technology Limited)
0015800001041RbAAI	Child Server 001	https://test.com	https://test.com	(Open Banking Limited (A))
0015800001041RbAAI	Child Server 002	https://test.com	https://test.com	(Open Banking Limited (A))
0015800001041RbAAI	Child AS 002	https://test.com	https://test.com	(Open 

In [4]:
import requests

print('List of ASPSPs that are active and have an Authorisation Server defined:')
count = 0
for resource in resources:
    name = resource['urn:openbanking:organisation:1.0']['OrganisationCommonName']
    ob_id = resource['urn:openbanking:organisation:1.0']['OBOrganisationId']
    has_auth = 'AuthorisationServers' in resource
    is_active = any(auth['Active'] for auth in resource["urn:openbanking:competentauthorityclaims:1.0"]['Authorisations'])
    
    if is_active and has_auth:
        for auth in resource['AuthorisationServers']:
            friendly_name = auth.get('CustomerFriendlyName')
            portal_url = auth.get('DeveloperPortalUri')
            config_url = auth.get('OpenIDConfigEndPointUri')
            
            auth_methods = []
            auth_url = None
            token_url = None
            register_url = None
            if config_url:
                try:
                    resp = requests.get(config_url, timeout=5)
                    resp.raise_for_status()
                    config = resp.json()
                    auth_methods = config.get('token_endpoint_auth_methods_supported', [])
                    auth_url = config.get('authorization_endpoint')
                    token_url = config.get('token_endpoint')
                    register_url = config.get('registration_endpoint')
                except Exception as ex: 
                    pass # print(ex)
            
            print(f"{ob_id}\t{friendly_name}\t{portal_url}\t{len(auth_methods)}\t{'Y' if register_url else 'N'}\t{'Y' if auth_url else 'N'}\t{'Y' if token_url else 'N'}\t({name})")
            count += 1

print(count, ' Auth servers')

List of ASPSPs that are active and have an Authorisation Server defined:
0015800001041RHAAY	EU Modelo Bank	https://o3bank.com	0	N	N	N	(Ozone Financial Technology Limited)
0015800001041RHAAY	EU Referenco Bank	https://o3bank.com	5	Y	Y	Y	(Ozone Financial Technology Limited)
0015800001041RHAAY	Ozone Bank 2	https://o3bank.com	4	Y	Y	Y	(Ozone Financial Technology Limited)
0015800001041RHAAY	Ozone - Bank 1	https://o3bank.com/	4	Y	Y	Y	(Ozone Financial Technology Limited)
0015800001041RbAAI	Child Server 001	https://test.com	0	N	N	N	(Open Banking Limited (A))
0015800001041RbAAI	Child Server 002	https://test.com	0	N	N	N	(Open Banking Limited (A))
0015800001041RbAAI	Child AS 002	https://test.com	0	N	N	N	(Open Banking Limited (A))
0015800001041RbAAI	Child 003	https://test.com	0	N	N	N	(Open Banking Limited (A))
0015800001041RbAAI	John - child	https://test.com	0	N	N	N	(Open Banking Limited (A))
0015800001041RbAAI	xxxxxxxxxxxxxxxxxxxxxxxxxxx	https://www.developerPortalURI.openbanking.org.uk	0	N	N	N	(Op

0015800001ZEc1gAAD	internal QA Optimum	https://obqa.pps-int.com/index.html	0	N	N	N	(PREPAY TECHNOLOGIES LIMITED)
0015800001ZEc1gAAD	internal QA Monese	https://obqa.pps-int.com/index.html	0	N	N	N	(PREPAY TECHNOLOGIES LIMITED)
0015800001ZEc1gAAD	internal DEV Optimum	https://ob-test.pps-int.com/client.html	0	N	N	N	(PREPAY TECHNOLOGIES LIMITED)
0015800001ZEc1gAAD	internal DEV Monese	https://ob-test.pps-int.com/client.html	0	N	N	N	(PREPAY TECHNOLOGIES LIMITED)
0015800001ZEc1gAAD	internal DEV QuidityPlus	https://ob-test.pps-int.com/client.html	0	N	N	N	(PREPAY TECHNOLOGIES LIMITED)
0015800001ZEc1gAAD	internal QA QuidityPlus	https://obqa.pps-int.com/index.html	0	N	N	N	(PREPAY TECHNOLOGIES LIMITED)
0015800001ZEc1gAAD	internal UAT QuidityPlus	https://obtest.pps-int.com/index.html	0	N	N	N	(PREPAY TECHNOLOGIES LIMITED)
0015800001ZEc1gAAD	internal UAT Optimum	https://obtest.pps-int.com/index.html	0	N	N	N	(PREPAY TECHNOLOGIES LIMITED)
0015800001ZEc1gAAD	internal UAT Monese	https://obtest.pps-int.com